In [13]:
import re

def ints_in_line(s):
    return list(map(int, re.findall(r"\d+", s)))

In [78]:
filename = "day5_input.txt"
# filename = "day5_example.txt"

with open(filename, "r") as f:
    lines = f.read().split("\n\n")
lines = [[s.strip() for s in l.split("\n") if s] for l in lines]

print(lines[:2])

[['seeds: 3139431799 50198205 3647185634 110151761 2478641666 139825503 498892555 8913570 961540761 489996751 568452082 100080382 907727477 42158689 1617552130 312026427 342640189 97088268 2049289560 336766062'], ['seed-to-soil map:', '1615836342 1401909974 23067952', '785532007 269485885 88937774', '3019002892 2773729385 10470414', '4202163101 2747292152 26437233', '3183210415 4217634159 77333137', '2847460091 3211730218 136699600', '2455891790 3791729773 70553041', '3260543552 2581343101 165949051', '3840286095 2849853212 361877006', '4228600334 2361239030 66366962', '1594559581 1077839137 21276761', '380069408 165017790 44262617', '3598718222 1894384162 241567873', '0 1424977926 190757551', '1894384162 2810496375 39356837', '424332025 606264721 196539291', '3521487829 2221977524 77230393', '742681934 69797365 36566707', '1638904294 1615735477 139190145', '1335949488 0 69797365', '779248641 802804012 6283366', '2638766896 4008940964 208693195', '250963029 1142644585 70452661', '19337

# Part 1

In [63]:
seeds = ints_in_line(lines[0][0])
lines = lines[1:]

In [65]:
def parse_map(ary):
    source, target = re.findall("(\w+)-to-(\w+) map:", ary[0])[0]
    triples = [ints_in_line(a) for a in ary[1:]]
    return (source, target, triples)

almanac = dict()
for line in lines:
    source, target, ranges = parse_map(line)
    d = dict()
    d['target'] = target
    d['ranges'] = [(s_start, t_start, span) for t_start, s_start, span in ranges]
    almanac[source] = d

In [67]:
def get_target_location(source_value, ranges):
    for s_start, t_start, span in ranges:
        if s_start <= source_value < s_start + span:
            return t_start + (source_value-s_start)
    return source_value

In [68]:
locations = []
for seed in seeds:
    current_value = seed
    current_type = 'seed'
    while current_type != 'location':
        current_value = get_target_location(current_value, almanac[current_type]['ranges'])
        current_type = almanac[current_type]['target']
    locations.append(current_value)
print(min(locations))

462648396


# Part 2

In [70]:
seed_pairs = []
temp_seeds = [s for s in seeds]
while temp_seeds:
    a,b = temp_seeds[:2]
    seed_pairs.append((a,b))
    temp_seeds = temp_seeds[2:]

In [71]:
seed_pairs

[(3139431799, 50198205),
 (3647185634, 110151761),
 (2478641666, 139825503),
 (498892555, 8913570),
 (961540761, 489996751),
 (568452082, 100080382),
 (907727477, 42158689),
 (1617552130, 312026427),
 (342640189, 97088268),
 (2049289560, 336766062)]

In [73]:
def get_target_location_ranges(source_start, source_span, ranges):
    lower_ranges = [r for r in ranges if r[0] <= source_start]
    if lower_ranges:
        max_lower_range = max(lower_ranges, key=lambda r: r[0])
        upper_end_of_range = max_lower_range[0] + max_lower_range[2]
    else:
        upper_end_of_range = 0

    if upper_end_of_range > source_start:
        if source_start+source_span <= upper_end_of_range:
            return [(source_start+max_lower_range[1]-max_lower_range[0], source_span)]
        else:
            return ([(source_start+max_lower_range[1]-max_lower_range[0], upper_end_of_range-source_start)]
                    + get_target_location_ranges(upper_end_of_range, source_span-(upper_end_of_range-source_start), ranges))

    upper_ranges = [r for r in ranges if r[0] >= source_start]
    if upper_ranges:
        min_upper_range = min(upper_ranges, key=lambda r: r[0])
        upper_end_of_range = min_upper_range[0] + min_upper_range[2]
    else:
        return [(source_start, source_span)]
    if min_upper_range[0] <= source_start + source_span:
        if source_start+source_span < upper_end_of_range:
            return [(source_start, min_upper_range[0]-source_start)] + [(min_upper_range[1], source_span-(min_upper_range[0]-source_start))]
        else:
            return [(source_start, min_upper_range[0]-source_start)] + [(min_upper_range[1], source_span-(min_upper_range[0]-source_start))] + get_target_location_ranges(upper_end_of_range, source_span-(upper_end_of_range-source_start), ranges)

    return [(source_start, source_span)]


In [74]:
current_ranges = seed_pairs
current_type = 'seed'
while current_type != 'location':
    new_ranges = []
    for source_start, source_span in current_ranges:
        new_ranges.extend(get_target_location_ranges(source_start, source_span, almanac[current_type]['ranges']))
    current_type = almanac[current_type]['target']
    current_ranges = new_ranges

In [81]:
print(f"number of locations: {sum([sp for _, sp in current_ranges])}")
print(f"lowest location: {min([st for st, _ in current_ranges])}")

number of locations: 1687205618
lowest location: 2520479
